<a href="https://colab.research.google.com/github/Nathalia-Prada/Sinergia_Bootcamp/blob/main/02_Caso_1_Book_Recommendation/Punto_3_Caso_1_Book_Recommendation_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Punto 3
3.	Implementación de un sistema de recomendación con Scikit-surprise


# Caso 1
# Book Recommendation Dataset


### 2024.11.12

## Download the dataframe from Kaggle
Install the kaggle python library

In [25]:
!pip install --user kaggle

Mount the Google drive so you can store your kaggle API credentials for future use

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:

import os

# Define the dataset file path
# Este es el nombre del file que estamos explorando en kaggle:
FILE = "arashnic/book-recommendation-dataset"

Make a directory for kaggle at the temporary instance location on Colab drive.

Download your kaggle API key (.json file). You can do this by going to your kaggle account page and clicking 'Create new API token' under the API section.

In [28]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


Upload the json file to Google Drive and then copy to the temporary location.

In [29]:
!cp /content/drive/MyDrive/Bootcamp/kaggle.json ~/.kaggle/kaggle.json

Change the file permissions to read/write to the owner only

In [30]:
! chmod 600 ~/.kaggle/kaggle.json

### Competitions and Datasets are the two types of Kaggle data

####1. Download competition data

If you get 403 Forbidden error, you need to click 'Late Submission' on the Kaggle page for that competition.

In [31]:
#! kaggle competitions download -c air-heart-disease

To downloaad specific files, instead of the entire data set

In [32]:

#! kaggle competitions download digit-recognizer -f train.csv

In [33]:
!kaggle datasets download -d $FILE

Dataset URL: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset
License(s): CC0-1.0
 70% 17.0M/24.3M [00:00<00:00, 172MB/s]
100% 24.3M/24.3M [00:00<00:00, 186MB/s]


#### 3. Unzip files if required

In [34]:
# Find the most recent file in the directory (assuming it is the downloaded file)
import glob
import zipfile

# Find the most recent file in the directory (assuming it is the downloaded file)
downloaded_files = glob.glob("*.zip")  # Looks for zip files in the current directory
if downloaded_files:
    latest_file = max(downloaded_files, key=os.path.getctime)  # Get the most recent file
    print(f"Downloaded file: {latest_file}")

    # Check if it's a zip file and unzip it
    if zipfile.is_zipfile(latest_file):
        print(f"Unzipping {latest_file}...")
        with zipfile.ZipFile(latest_file, 'r') as zip_ref:
            zip_ref.extractall()  # Extract to the current directory
        print(f"{latest_file} unzipped successfully.")
        !rm *.zip
    else:
        print(f"{latest_file} is not a zip file.")
else:
    print("No zip file found.")

Downloaded file: book-recommendation-dataset.zip
Unzipping book-recommendation-dataset.zip...
book-recommendation-dataset.zip unzipped successfully.


#### 4. Load the CSV dataset in a Pandas Dataframe

In [35]:

import pandas as pd
import glob

# Find all CSV files in the current directory and subdirectories
csv_files = glob.glob('**/Ratings.csv', recursive=True)

# Check if train.csv file was found
if csv_files:
    # Load the first found train.csv file into a DataFrame
    df = pd.read_csv(csv_files[0])

    # Display the info of the DataFrame
    print(f"DataFrame loaded from: {csv_files[0]}")
    print(df.info())
else:
    print("No CSV files found in the current directory.")

DataFrame loaded from: Ratings.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None


In [36]:
df.isnull().sum()

,0
User-ID,0
ISBN,0
Book-Rating,0


In [37]:
# Percentage of missing values for each column
missing_percentage = (df.isnull().sum() / len(df)) * 100

# Display the result
print(missing_percentage)

User-ID        0.0
ISBN           0.0
Book-Rating    0.0
dtype: float64


# Data Cleaning

In [38]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
User-ID,1149780.0,NaN,NaN,NaN,140386.395126,80562.277718,2.0,70345.0,141010.0,211028.0,278854.0
ISBN,1149780,340556,0971880107,2502,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Book-Rating,1149780.0,NaN,NaN,NaN,2.86695,3.854184,0.0,0.0,0.0,7.0,10.0
